In [27]:
from langchain.agents.structured_output import ToolStrategy
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_groq import ChatGroq
from langchain_core.tools import Tool
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from pydantic import BaseModel, Field
from typing import List, Optional
import os
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
ddg_search = DuckDuckGoSearchResults()

ddg_search_tool = Tool(
    name="DuckDuckGoSearch",
    func=ddg_search.run,  # Uses the standard `.run()` interface
    description=(
        "Use this tool to perform a DuckDuckGo web search and return JSON-formatted results. "
        "Input: a search query string; Output: a JSON array of search results."
    )
)

In [6]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [31]:
model_name = "qwen/qwen3-32b" #"moonshotai/kimi-k2-instruct" "qwen/qwen3-32b"
temperature = 0.0


model = ChatGroq(
    model_name=model_name,
    temperature=temperature
)   


In [32]:
# Define your structured output schema
class SearchResult(BaseModel):
    """Structured search result"""
    title: str = Field(description="Title of the result")
    url: str = Field(description="URL of the result")
    snippet: str = Field(description="Brief snippet from the result")

class AgentResponse(BaseModel):
    """Structured agent response"""
    answer: str = Field(description="The main answer to the query")
    sources: List[SearchResult] = Field(description="Sources used", default_factory=list)
    confidence: Optional[str] = Field(description="Confidence level", default=None)


In [33]:
# Create the agent
memory = MemorySaver()

In [34]:
# Create agent with structured model
agent = create_agent(model, 
                    tools=[ddg_search_tool], 
                    checkpointer=memory,
                    response_format=ToolStrategy(AgentResponse),
                    system_prompt="You are a helpful assistant. Be concise and accurate.")

In [37]:
# Use the agent
config = {"configurable": {"thread_id": "xyz123"}}

input = {"messages": [{"role": "user", "content": "What is the capital of France?"}]}

result = agent.invoke(input, config=config)


In [42]:
structured_response: AgentResponse = result["structured_response"]

# Access fields
answer = structured_response.answer
confidence = structured_response.confidence
sources = structured_response.sources

print(answer)
print(confidence)
print(sources)

The capital of France is Paris.
high
[SearchResult(title='Paris - Wikipedia', url='https://en.wikipedia.org/wiki/Paris', snippet='Paris, then already the capital of France, was the most populous city of Europe.'), SearchResult(title='Current time in Paris, France', url='https://24timezones.com/Paris/time', snippet='Paris is the capital of France.'), SearchResult(title='Time in Paris, France now', url='https://time.is/Paris', snippet='Paris is the capital of France.')]
